In this notebook I'll explore how to run normal (pooled) OLS, Fixed Effects, and Random Effects in Python, R, and Stata.

# OLS

This example is taken from here: http://stackoverflow.com/questions/19991445/run-an-ols-regression-with-pandas-data-frame

## Regular API

I'll start by using the regular API.


In [1]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import linearmodels
from statsmodels.sandbox.regression.predstd import wls_prediction_std

np.random.seed(9876789)

In [2]:
%load_ext rpy2.ipython

In [3]:
nsample = 100
x = np.linspace(0, 10, 100)
X = np.column_stack((x, x**2))
beta = np.array([1, 0.1, 10])
e = np.random.normal(size=nsample)

In [4]:
X = sm.add_constant(X)
y = np.dot(X, beta) + e

In [5]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 4.020e+06
Date:                Wed, 23 May 2018   Prob (F-statistic):          2.83e-239
Time:                        04:45:54   Log-Likelihood:                -146.51
No. Observations:                 100   AIC:                             299.0
Df Residuals:                      97   BIC:                             306.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3423      0.313      4.292      0.0

### Formula API

Now we use the `formula` API.

In [6]:
import pandas as pd
import statsmodels.formula.api as smf
df = pd.DataFrame({"A": [10,20,30,40,50], "B": [20, 30, 10, 40, 50], "C": [32, 234, 23, 23, 42523]})
df

,A,B,C
0,10,20,32
1,20,30,234
2,30,10,23
3,40,40,23
4,50,50,42523


In [7]:
result = smf.ols(formula="A ~ B + C", data=df).fit()
result.params

Intercept    14.952480
B             0.401182
C             0.000352
dtype: float64

In [8]:
result.summary()

/home/vagrant/anaconda/lib/python3.6/site-packages/statsmodels/stats/stattools.py:72: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  "samples were given." % int(n), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      A   R-squared:                       0.579
Model:                            OLS   Adj. R-squared:                  0.158
Method:                 Least Squares   F-statistic:                     1.375
Date:                Wed, 23 May 2018   Prob (F-statistic):              0.421
Time:                        04:45:54   Log-Likelihood:                -18.178
No. Observations:                   5   AIC:                             42.36
Df Residuals:                       2   BIC:                             41.19
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     14.9525     17.764      0.842      0.489     -61.481      91.386
B              0.4012      0.650      0.617      0.600      -2.394       3.197
C              0.0004      0.001      0.650      0.583      -0.002       0.003
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.061
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.498
Skew:                          -0.123   Prob(JB):                        0.780
Kurtosis:                       1.474   Cond. No.                     5.21e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.21e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
df

,A,B,C
0,10,20,32
1,20,30,234
2,30,10,23
3,40,40,23
4,50,50,42523


In [10]:
%%R -i df
df

   A  B     C
0 10 20    32
1 20 30   234
2 30 10    23
3 40 40    23
4 50 50 42523


In [11]:
%%R 
fit = lm(A ~ B + C, df)
fit


Call:
lm(formula = A ~ B + C, data = df)

Coefficients:
(Intercept)            B            C  
  1.495e+01    4.012e-01    3.516e-04  



In [12]:
%%R
summary(fit)


Call:
lm(formula = A ~ B + C, data = df)

Residuals:
        0         1         2         3         4 
-12.98738  -7.07022  11.02761   8.99214   0.03785 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)
(Intercept) 1.495e+01  1.776e+01   0.842    0.489
B           4.012e-01  6.497e-01   0.617    0.600
C           3.516e-04  5.412e-04   0.650    0.583

Residual standard error: 14.51 on 2 degrees of freedom
Multiple R-squared:  0.5789,	Adjusted R-squared:  0.1577 
F-statistic: 1.375 on 2 and 2 DF,  p-value: 0.4211



# Fixed Effects

Here I'll test out fixed effects estimation. In particular, I want to compare the outcomes that I get in Python to the output that I get from Stata. I want to make sure I'm getting the same results. To do this, I'll be using a data exercise from Wooldridge's panel data book. I have solutions to the odd problems in this book and Stata results. So, I can verify that my Stata results are accurate and then proceed with the analysis in Python.


## Linear Mixed Effects Models


#### In `statsmodels`
Here is a quick example of the syntax from `statsmodels`. This is taken from here: http://statsmodels.sourceforge.net/devel/mixed_linear.html

In [13]:
import statsmodels.api as sm 
import statsmodels.formula.api as smf

data = sm.datasets.get_rdataset("dietox", "geepack").data
data.head()

,Weight,Feed,Time,Pig,Evit,Cu,Litter
0,26.50000,NaN,1,4601,1,1,1
1,27.59999,5.200005,2,4601,1,1,1
2,36.50000,17.600000,3,4601,1,1,1
3,40.29999,28.500000,4,4601,1,1,1
4,49.09998,45.200001,5,4601,1,1,1


In [14]:
data.describe()

,Weight,Feed,Time,Pig,Evit,Cu,Litter
count,861.000000,789.000000,861.000000,861.000000,861.000000,861.000000,861.000000
mean,60.725769,80.728645,6.480836,6238.319396,2.026713,2.015099,12.135889
std,24.978881,52.877736,3.444735,1323.845928,0.817246,0.807525,7.427252
min,15.000000,3.300003,1.000000,4601.000000,1.000000,1.000000,1.000000
25%,38.299990,32.800003,3.000000,4857.000000,1.000000,1.000000,5.000000
50%,59.199980,74.499996,6.000000,5866.000000,2.000000,2.000000,11.000000
75%,81.199950,123.000000,9.000000,8050.000000,3.000000,3.000000,20.000000
max,117.000000,224.500000,12.000000,8442.000000,3.000000,3.000000,24.000000


In [15]:
md = smf.mixedlm("Weight ~ Time", data, groups=data["Pig"]) 
mdf = md.fit() 
mdf.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:            MixedLM Dependent Variable: Weight    
No. Observations: 861     Method:             REML      
No. Groups:       72      Scale:              11.3669   
Min. group size:  11      Likelihood:         -2404.7753
Max. group size:  12      Converged:          Yes       
Mean group size:  12.0                                  
--------------------------------------------------------
             Coef.  Std.Err.    z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept    15.724    0.788  19.952 0.000 14.179 17.268
Time          6.943    0.033 207.939 0.000  6.877  7.008
Group Var    40.394    2.149                            
========================================================

"""

In [16]:
md = smf.MixedLM.from_formula("Weight ~ Time", data, groups=data["Pig"]) 
mdf = md.fit() 
mdf.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:            MixedLM Dependent Variable: Weight    
No. Observations: 861     Method:             REML      
No. Groups:       72      Scale:              11.3669   
Min. group size:  11      Likelihood:         -2404.7753
Max. group size:  12      Converged:          Yes       
Mean group size:  12.0                                  
--------------------------------------------------------
             Coef.  Std.Err.    z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept    15.724    0.788  19.952 0.000 14.179 17.268
Time          6.943    0.033 207.939 0.000  6.877  7.008
Group Var    40.394    2.149                            
========================================================

"""

#### In Stata

Now I'll confirm that I get the same calculation in Stata.

In [17]:
#Save file as Stata file
# data.to_stata('lme.dta')

In Stata, call the following commands:

    use "C:\Users\Jeremy\Documents\GitRepositories\Learn\python\statistics-and-data-analysis\General\fixed-and-random-effects\lme.dta", clear
    
    xtreg Weight Time, i(Pig)
    
This produces the following output:

#### With `plm` in R

Some infor on how to do this in R: https://dss.princeton.edu/training/Panel101R.pdf

In [18]:
%%R
library(foreign)
library(plm)

/home/vagrant/anaconda/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required package: Formula

  warnings.warn(x, RRuntimeWarning)


In [19]:
%%R
df = read.dta('lme.dta')
fixed = plm(Weight ~ Time, data=df, index=c('Pig'), model='within')
summary(fixed)

Oneway (individual) effect Within Model

Call:
plm(formula = Weight ~ Time, data = df, model = "within", index = c("Pig"))

Unbalanced Panel: n=72, T=11-12, N=861

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
-15.900  -1.920  -0.284   1.650  16.400 

Coefficients :
     Estimate Std. Error t-value  Pr(>|t|)    
Time 6.942398   0.033388  207.93 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    500420
Residual Sum of Squares: 8957.2
R-Squared:      0.9821
Adj. R-Squared: 0.98047
F-statistic: 43236.1 on 1 and 788 DF, p-value: < 2.22e-16


As you can see, the results are the same.

## Wooldridge Panel Data: Chapter 10


I will replicate the examples that are found here: http://www.ats.ucla.edu/stat/stata/examples/eacspd/chapter10.htmb (Now here: https://stats.idre.ucla.edu/stata/examples/eacspd/econometric-analysis-of-cross-section-and-panel-data-by-jeffrey-m-wooldridgechapter-10-basic-linear-unobserved-effects-panel-data-models/) 

**jtrain1 data**

This data comes from the paper "Are Training Subsidies for Firms Effective? The Michigan Experience", by Holzer, et al. 1993. 

Abstract:
> This paper explores the effects of a state-financed training grant program for manufacturing firms in Michigan. Using a three-year panel of data from a unique survey of firms that applied for these grants, the authors estimate the effects of receipt of a grant on total hours of training in the firm and the product scrap rate. They find that receipt of these grants is associated with a large and significant, though one-time, increase in training hours, and with a more lasting reduction in scrap rates.


### Random Effects Model

#### Example 10.4 on page 261 using jtrain1.dta.


> We now use the data in JTRAIN1.RAW to estimate the effect of job training grants on firm scrap rates, using a random e¤ects analysis. There are 54 firms that reported scrap rates for each of the years 1987, 1988, and 1989. Grants were not awarded in 1987. Some firms received grants in 1988, others received grants in 1989, and a firm could not receive a grant twice. Since there are firms in 1989 that received a grant only in 1988, it is important to allow the grant effect to persist one period. The estimated equation is

![Example_10_4_Wooldridge_Panel](./Example_10_4_Wooldridge_Panel.png)


In this example, I'm trying to replicate the following Stata code.

    //use jtrain1, clear
    use http://www.stata.com/data/jwooldridge/eacsap/jtrain1, clear
    xtreg lscrap d88 d89 union grant grant_1, i( fcode)

In [20]:
import pandas as pd
import pandas
import statsmodels.formula.api as smf

In [21]:
# #download and save the correct datasets
# import urllib
# testfile = urllib.request.URLopener()
# testfile.retrieve("http://www.stata.com/data/jwooldridge/eacsap/jtrain1.dta", "jtrain1.dta")

Note that once you download this file, you need to use Stata to save this in Stata 11/12 format or it will not work. I have already done this and named the new copy of the file

In [22]:
# Does not work with this version of Stata
# df = pd.read_stata('jtrain1.dta')

In [23]:
#Open converted file
url = "http://www.stata.com/data/jwooldridge/eacsap/jtrain1.dta"
# df = pd.read_stata('St11-12_jtrain1.dta')
df = pd.read_stata(url)

In [24]:
# The Stata Reader doesn't work with all versions of Stata files
# This Stata file has been converted to one that it can read.
with open('St11-12_jtrain1.dta', 'rb') as file:
    stata_file = sm.iolib.foreign.StataReader(file)

# Print Variables names and labels
for var in stata_file.variables():
    print(f'{var.name}: {var.label}')

year: 1987, 1988, or 1989
fcode: firm code number
employ: # employees at plant
sales: annual sales, $
avgsal: average employee salary
scrap: scrap rate (per 100 items)
rework: rework rate (per 100 items)
tothrs: total hours training
union: =1 if unionized
grant: = 1 if received grant
d89: = 1 if year = 1989
d88: = 1 if year = 1988
totrain: total employees trained
hrsemp: tothrs/totrain
lscrap: log(scrap)
lemploy: log(employ)
lsales: log(sales)
lrework: log(rework)
lhrsemp: log(1 + hrsemp)
lscrap_1: lagged lscrap; missing 1987
grant_1: lagged grant; assumed 0 in 1987
clscrap: lscrap - lscrap_1; year > 1987
cgrant: grant - grant_1
clemploy: lemploy - lemploy[_n-1]
clsales: lavgsal - lavgsal[_n-1]
lavgsal: log(avgsal)
clavgsal: lavgsal - lavgsal[_n-1]
cgrant_1: cgrant[_n-1]
chrsemp: hrsemp - hrsemp[_n-1]
clhrsemp: lhrsemp - lhrsemp[_n-1]


In [25]:
#df = df.set_index(['year'])
df.head()

,year,fcode,employ,sales,avgsal,scrap,rework,tothrs,union,grant,...,grant_1,clscrap,cgrant,clemploy,clsales,lavgsal,clavgsal,cgrant_1,chrsemp,clhrsemp
0,1987,410032.0,100.0,47000000.0,35000.0,NaN,NaN,12.0,0,0,...,0,NaN,0,NaN,NaN,10.463103,NaN,NaN,NaN,NaN
1,1988,410032.0,131.0,43000000.0,37000.0,NaN,NaN,8.0,0,0,...,0,NaN,0,0.270027,-0.088949,10.518673,0.055570,0.0,-8.946565,-1.165385
2,1989,410032.0,123.0,49000000.0,39000.0,NaN,NaN,8.0,0,0,...,0,NaN,0,-0.063013,0.130621,10.571317,0.052644,0.0,0.198597,0.047832
3,1987,410440.0,12.0,1560000.0,10500.0,NaN,NaN,12.0,0,0,...,0,NaN,0,NaN,NaN,9.259130,NaN,NaN,NaN,NaN
4,1988,410440.0,13.0,1970000.0,11000.0,NaN,NaN,12.0,0,0,...,0,NaN,0,0.080043,0.233347,9.305651,0.046520,0.0,0.000000,0.000000


In [26]:
df[['lscrap', 'union', 'fcode','sales', 'rework']].describe()

,lscrap,union,fcode,sales,rework
count,162.000000,471.000000,471.000000,373.0,123.000000
mean,0.393681,0.197452,415709.000000,6116037.0,3.473984
std,1.486471,0.398500,4022.922363,7912517.0,5.462482
min,-4.605170,0.000000,410032.000000,110000.0,0.000000
25%,-0.523431,0.000000,410604.000000,1550000.0,0.350000
50%,0.347123,0.000000,418084.000000,3000000.0,1.160000
75%,1.386294,0.000000,419309.000000,7700000.0,4.000000
max,3.401197,1.000000,419486.000000,54000000.0,40.000000


#### With `statsmodels`

I believe that this is working because it doesn't know how to handle the missing data. I'm going to drop all the missing data and then try again.

In [27]:
# The following wont work if you don't drop the missing values.
df2 = df.copy()

# df2 = df[['lscrap', 'd88', 'd89', 'union', 'grant', 'grant_1', 'fcode']].dropna()

In [28]:
# The following wont work if you don't drop the missing values.

# md = smf.mixedlm("lscrap ~ d88 + d89 + union + grant + grant_1", df2, groups=df2["fcode"])
# mdf = md.fit() 
# mdf.summary()

In [29]:
# The following wont work if you don't drop the missing values.

# md = smf.mixedlm("lscrap ~ union", df2, groups=df2["fcode"])
# mdf = md.fit() 
# mdf.summary()

In [30]:
df2 = df[['lscrap', 'd88', 'd89', 'union', 'grant', 'grant_1', 'fcode']].dropna()
md = smf.mixedlm("lscrap ~ d88 + d89 + union + grant + grant_1", df2, groups=df2["fcode"])
mdf = md.fit() 
mdf.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
=======================================================
Model:            MixedLM Dependent Variable: lscrap   
No. Observations: 162     Method:             REML     
No. Groups:       54      Scale:              0.2478   
Min. group size:  3       Likelihood:         -205.9266
Max. group size:  3       Converged:          Yes      
Mean group size:  3.0                                  
-------------------------------------------------------
             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.415    0.244  1.698 0.089 -0.064  0.894
d88          -0.093    0.109 -0.856 0.392 -0.307  0.120
d89          -0.270    0.132 -2.050 0.040 -0.527 -0.012
union         0.548    0.412  1.329 0.184 -0.260  1.356
grant        -0.215    0.148 -1.455 0.146 -0.505  0.075
grant_1      -0.378    0.205 -1.839 0.066 -0.780  0.025
Group Var     1.955    0.984                           
=======================================================

"""

#### With `linearmodels`

In [31]:
df2 = df[['lscrap', 'd88', 'd89', 'union', 'grant', 'grant_1', 'fcode', 'year']].dropna().set_index(['fcode', 'year'])
md = linearmodels.RandomEffects.from_formula("lscrap ~ 1+ d88 + d89 + union + grant + grant_1 + EntityEffects", data=df2)
mdf = md.fit() 
mdf

Dep. Variable:,lscrap,R-squared:,0.1486
Estimator:,RandomEffects,R-squared (Between):,0.0184
No. Observations:,162,R-squared (Within):,0.2005
Date:,"Wed, May 23 2018",R-squared (Overall):,0.0349
Time:,04:46:10,Log-likelihood,-113.26
Cov. Estimator:,Unadjusted,,
,,F-statistic:,5.4474
Entities:,54,P-value,0.0001
Avg Obs:,3.0000,Distribution:,"F(5,156)"
Min Obs:,3.0000,,
Max Obs:,3.0000,F-statistic (robust):,5.4474


#### With `plm` in R

Some info on how to do this in R: https://dss.princeton.edu/training/Panel101R.pdf

In [32]:
dfr = df2.reset_index()

In [33]:
%%R -i dfr
reg = plm(lscrap ~ d88 + d89 + union + grant + grant_1, data=dfr, index=c('fcode'), model='random')
summary(reg)

Oneway (individual) effect Random Effect Model 
   (Swamy-Arora's transformation)

Call:
plm(formula = lscrap ~ d88 + d89 + union + grant + grant_1, data = dfr, 
    model = "random", index = c("fcode"))

Balanced Panel: n=54, T=3, N=162

Effects:
                 var std.dev share
idiosyncratic 0.2502  0.5002 0.114
individual    1.9347  1.3909 0.886
theta:  0.7967  

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
-2.5500 -0.2250  0.0403  0.2570  1.5500 

Coefficients :
             Estimate Std. Error t-value Pr(>|t|)  
(Intercept)  0.414837   0.242837  1.7083  0.08957 .
d88         -0.093553   0.109299 -0.8559  0.39335  
d89         -0.270005   0.131813 -2.0484  0.04220 *
union        0.547792   0.409536  1.3376  0.18298  
grant       -0.214409   0.147959 -1.4491  0.14931  
grant_1     -0.376735   0.205588 -1.8325  0.06879 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    45.616
Residual Sum of Squares: 38.902
R-Squared:      0.14718

If I don't include the index, not that R still understands the
Panel structure:

    Balanced Panel: n=54, T=3, N=162

That is because it simply assumes that the
first column is the entity and the second is the time variable. Also, notice that the default is entity effects.

In [34]:
%%R -i dfr
# This works too.
reg = plm(lscrap ~ d88 + d89 + union + grant + grant_1, data=dfr, model='random')
summary(reg)

Oneway (individual) effect Random Effect Model 
   (Swamy-Arora's transformation)

Call:
plm(formula = lscrap ~ d88 + d89 + union + grant + grant_1, data = dfr, 
    model = "random")

Balanced Panel: n=54, T=3, N=162

Effects:
                 var std.dev share
idiosyncratic 0.2502  0.5002 0.114
individual    1.9347  1.3909 0.886
theta:  0.7967  

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
-2.5500 -0.2250  0.0403  0.2570  1.5500 

Coefficients :
             Estimate Std. Error t-value Pr(>|t|)  
(Intercept)  0.414837   0.242837  1.7083  0.08957 .
d88         -0.093553   0.109299 -0.8559  0.39335  
d89         -0.270005   0.131813 -2.0484  0.04220 *
union        0.547792   0.409536  1.3376  0.18298  
grant       -0.214409   0.147959 -1.4491  0.14931  
grant_1     -0.376735   0.205588 -1.8325  0.06879 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    45.616
Residual Sum of Squares: 38.902
R-Squared:      0.14718
Adj. R-Squared: 0.1

In [35]:
%%R -i dfr
# I'm not sure why this doesn't work.
# reg = plm(lscrap ~ union + grant + grant_1, data=dfr, index=c('fcode', 'year'), model='random', effect='twoways')
# summary(reg)

NULL


#### In Stata

Compare this to the Stata output here. They are the same

```
//use jtrain1, clear
use http://www.stata.com/data/jwooldridge/eacsap/jtrain1, clear
xtreg lscrap d88 d89 union grant grant_1, i( fcode)
```

Below I give the Stata output when `robust` is used. The regular procedure above seems to be closer to the Python output.

As can be read here (http://statsmodels.sourceforge.net/devel/generated/statsmodels.regression.mixed_linear_model.MixedLM.from_formula.html), "this method currently does not correctly handle missing values, so missing values should be explicitly dropped from the DataFrame before calling this method."

## Fixed Effects


### Example 10.5 on page 272 using jtrain1.dta.


![10_5](./Example_10_5_Wooldridge_Panel.png)

In this example, I'm trying to replicate the following Stata code.

    //use jtrain1, clear
    use http://www.stata.com/data/jwooldridge/eacsap/jtrain1, clear
    xtreg lscrap d88 d89 union grant grant_1, i( fcode) fe
    
To do this, I'm going to use the direction that is given here: http://stackoverflow.com/questions/24195432/fixed-effect-in-pandas-or-statsmodels

#### Using `linearmodels`

In [36]:
import pandas as pd
import pandas
import statsmodels.formula.api as smf
df = pd.read_stata('St11-12_jtrain1.dta')

In [37]:
df.columns

Index(['year', 'fcode', 'employ', 'sales', 'avgsal', 'scrap', 'rework',
       'tothrs', 'union', 'grant', 'd89', 'd88', 'totrain', 'hrsemp', 'lscrap',
       'lemploy', 'lsales', 'lrework', 'lhrsemp', 'lscrap_1', 'grant_1',
       'clscrap', 'cgrant', 'clemploy', 'clsales', 'lavgsal', 'clavgsal',
       'cgrant_1', 'chrsemp', 'clhrsemp'],
      dtype='object')

In [38]:
df2.head()

lscrap  d88  d89  union  grant  grant_1
fcode    year                                           
410523.0 1987 -2.813411    0    0      0      0        0
         1988 -2.995732    1    0      0      0        0
         1989 -2.995732    0    1      0      0        0
410538.0 1987  0.970779    0    0      1      0        0
         1988  1.007958    1    0      1      0        0

In [39]:
df2 = df[['lscrap', 'union', 'grant', 'grant_1', 'fcode', 'year']].dropna().set_index(['fcode', 'year'])
reg = linearmodels.PanelOLS.from_formula("lscrap ~ grant + grant_1 + EntityEffects + TimeEffects", data=df2)
# This wont work below since the variable 'union' doesn't vary over time for
# any of the firms in the sample
# reg = linearmodels.PanelOLS.from_formula("lscrap ~ union + grant + grant_1 + EntityEffects + TimeEffects", data=df2)
reg_fitted = reg.fit() 
reg_fitted

Dep. Variable:,lscrap,R-squared:,0.0411
Estimator:,PanelOLS,R-squared (Between):,-0.0552
No. Observations:,162,R-squared (Within):,0.1478
Date:,"Wed, May 23 2018",R-squared (Overall):,-0.0381
Time:,04:46:12,Log-likelihood,-80.946
Cov. Estimator:,Unadjusted,,
,,F-statistic:,2.2294
Entities:,54,P-value,0.1127
Avg Obs:,3.0000,Distribution:,"F(2,104)"
Min Obs:,3.0000,,
Max Obs:,3.0000,F-statistic (robust):,2.2294


In [40]:
# # This doesn't give the correct results
# df2 = df[['d88', 'd89', 'lscrap', 'union', 'grant', 'grant_1', 'fcode', 'year']].dropna()
# reg = smf.mixedlm("lscrap ~ d88 + d89 + grant + grant_1", df2, groups=df2["fcode"])
# reg_fitted = reg.fit()
# reg_fitted.summary()

In [41]:
# df2 = df.set_index(['year', 'fcode'])
# reg = pd.stats.plm.PanelOLS(y=df2['lscrap'], x=df2[['d88', 'd89', 'union', 'grant', 'grant_1']], time_effects=True)
# reg

Note that the indexes must be in the correct order. The above results do not match the Stata output. Below is the correct ordering.

In [42]:
# df2 = df.set_index(['fcode', 'year'])
# reg = pd.stats.plm.PanelOLS(y=df2['lscrap'], x=df2[['d88', 'd89', 'union', 'grant', 'grant_1']], time_effects=True)
# reg

#### Using Stata

Compare this to the Stata output. It gives the same coefficients. Some of the tests are not the same, however.

#### Using `plm` in R

In [43]:
%%R -i df
reg = plm(lscrap ~ d88 + d89 + union + grant + grant_1, df, index=c('fcode', 'year'), model='within')
summary(reg)

Oneway (individual) effect Within Model

Call:
plm(formula = lscrap ~ d88 + d89 + union + grant + grant_1, data = df, 
    model = "within", index = c("fcode", "year"))

Balanced Panel: n=54, T=3, N=162

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
-2.2900 -0.1120 -0.0178  0.1440  1.4300 

Coefficients :
         Estimate Std. Error t-value Pr(>|t|)  
d88     -0.080216   0.109475 -0.7327  0.46537  
d89     -0.247203   0.133218 -1.8556  0.06634 .
grant   -0.252315   0.150629 -1.6751  0.09692 .
grant_1 -0.421590   0.210200 -2.0057  0.04749 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    32.25
Residual Sum of Squares: 25.766
R-Squared:      0.20105
Adj. R-Squared: -0.23684
F-statistic: 6.54259 on 4 and 104 DF, p-value: 9.7741e-05


If we don't want to put in time dummies manually, all we need to do
is to specify "two-way" fixed effects. Note that we need to supply the 
index properly. That is, the individual index first, followed by the time
index second. If you don't specify the index, it will assume that the first
two columns are the index, again in the order of individual and then time.

See p. 72 of this: https://cran.r-project.org/web/packages/plm/plm.pdf and see p. 5 of this https://cran.r-project.org/web/packages/plm/vignettes/plm.pdf

In [44]:
%%R -i df
# This works so that we don't have to manually put in time dummies
reg = plm(lscrap ~ union + grant + grant_1, df, index=c('fcode', 'year'), model='within', effect='twoways')
summary(reg)

Twoways effects Within Model

Call:
plm(formula = lscrap ~ union + grant + grant_1, data = df, effect = "twoways", 
    model = "within", index = c("fcode", "year"))

Balanced Panel: n=54, T=3, N=162

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
-2.2900 -0.1120 -0.0178  0.1440  1.4300 

Coefficients :
        Estimate Std. Error t-value Pr(>|t|)  
grant   -0.25231    0.15063 -1.6751  0.09692 .
grant_1 -0.42159    0.21020 -2.0057  0.04749 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    26.871
Residual Sum of Squares: 25.766
R-Squared:      0.041111
Adj. R-Squared: -0.48443
F-statistic: 2.22942 on 2 and 104 DF, p-value: 0.11271


A simple Monte Carlo demonstration using a different package can be found here: https://www.r-bloggers.com/linear-models-with-multiple-fixed-effects/